In [0]:
import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage
import lsst.afw.geom as afwGeom
import numpy as np
import os
import lsst.utils
import pfs.drp.stella.findAndTraceAperturesTask as fataTask
import pfs.drp.stella as drpStella
try:
    import lsst.afw.display as afwDisplay
except ImportError:
    afwDisplay = None

if afwDisplay:
    try:
        afwDisplay.setDefaultBackend("ds9" if True else "virtualDevice")
    except RuntimeError as e:
        print e

    afwDisplay.setDefaultMaskTransparency(75)

In [0]:
testDataDir = os.path.join(lsst.utils.getPackageDir("drp_stella_data"),'data/PFS')
butler = dafPersist.Butler(testDataDir)

In [0]:
dataId = dict(site='S', category='A', filter='r', visit=5, ccd=5)
exposure = butler.get('postISRCCD', dataId)
print exposure

In [0]:
display0 = afwDisplay.getDisplay()
display0.mtv(exposure, title="parent")
display0.setMaskTransparency(50)
display0.setMaskPlaneColor("CROSSTALK", "orange")

In [0]:
print dir(exposure)

In [0]:
myFindTask = fataTask.FindAndTraceAperturesTask()
fts = myFindTask.run(exposure)
print fts.size()

In [0]:
for iTrace in range(fts.size()):
    ft = fts.getFiberTrace(iTrace)
    xCenters = ft.getXCenters()
    ftFunction = ft.getFiberTraceFunction()
    yLow = ftFunction.yCenter + ftFunction.yLow
    yHigh = ftFunction.yCenter + ftFunction.yHigh

    for y in np.arange(yLow, yHigh+1, 20):
        point = afwGeom.Point2D(xCenters[y-yLow],y)
        display0.dot("+", xCenters[y-yLow], y)

In [0]:
for ft in fts.getTraces():
    drpStella.markFiberTraceInMask(ft, exposure.getMaskedImage())

In [0]:
display0 = afwDisplay.getDisplay()
display0.mtv(exposure, title="parent")
display0.setMaskTransparency(50)
display0.setMaskPlaneColor("CROSSTALK", "orange")